ICEL 的全称是 LangChain Expression Language，是 LangChain 0.1.0+ 推出的全新核心架构，它的设计目标是让链（Chain）更像「表达式组合」：模块之间可以像拼积木一样组合，但又能高度并发、异步、可追踪、可部署。。这是一种 LangChain 优化链的做法，在 Runtime 执行。我们经常把 ICEL 创建的 `Runnable` 称作**链（Chain）**。官方文档参考[这里](https://python.langchain.com/docs/concepts/lcel/)。

第一点就是 [RunnableSequence](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableSequence.html)，这种用法是构建 `Runnable` 序列，其中每个的输出是下一个的输入。

`RunnableSequence` 可以直接实例化，或者更常见的是使用 `|` 运算符来实例化，其中左操作数或右操作数（或两者）必须是 `Runnable`。我们写一个简单的例子。

或者是各种各样的别的写法：
```python
chain = RunnableSequence([runnable1, runnable2])
chain = runnable1.pipe(runnable2)
```

In [13]:
from langchain_core.runnables import RunnableLambda, RunnableSequence


def add_one(x: int) -> int:
    """Given a variate x, return (x + 1)"""
    return x + 1


def mul_two(x: int) -> int:
    """Given a variate x, return (x * 2)"""
    return x * 2


def mul_three(x: int) -> int:
    """Given a variate x, return (x * 3)"""
    return x * 3


def mul_four(x: int) -> int:
    """Given a variate x, return (x * 4)"""
    return x * 4


runnable_op_1 = RunnableLambda(add_one)
runnable_op_2 = RunnableLambda(mul_two)
runnable_op_3 = RunnableLambda(mul_three)
runnable_op_4 = RunnableLambda(mul_four)

# 可以看到源代码，包含三个可执行步骤
sequence = RunnableSequence(first=runnable_op_1, last=runnable_op_2)
print("Given 1, add_one then mul_two:", sequence.invoke(1))
print("Given [1, 2, 3], add_one then mul_two:", await sequence.abatch([1, 2, 3]))

print("\n===================================\n")
# 我们测试换一个顺序，可以看见结果就会不一样
sequence = RunnableSequence(first=runnable_op_2, last=runnable_op_1)
print("Given 1, mul_two then add_one:", sequence.invoke(1))

print("\n===================================\n")
# 我们使用重载之后的 | 运算符
sequence = runnable_op_1 | runnable_op_2
print("Given 1, add_one then mul_two:", sequence.invoke(1))

print("\n===================================\n")
# 试一试超出三个运算，我们来到四个运算
sequence = runnable_op_1 | runnable_op_2 | runnable_op_3 | runnable_op_4
print("Given 1, add_one, mul_two, mul_three then mul_four:", sequence.invoke(1))

Given 1, add_one then mul_two: 4
Given [1, 2, 3], add_one then mul_two: [4, 6, 8]


Given 1, mul_two then add_one: 3


Given 1, add_one then mul_two: 4


Given 1, add_one, mul_two, mul_three then mul_four: 48


1. 细节问题：其实按道理我们得包装我们的函数，使用 `RunnableLambda(some_func)`，但是在底层有一些广播的优化，可以自动将我们的函数转换成 `Runnable`。但是必须小心 lambda 函数不是一个 `RunnableLambda`对象，它只是一个函数。

2. 源码解析：可以在 Chain 中叠加三层以上的运算吗？
```python
def __init__(
        self,
        *steps: RunnableLike,
        name: Optional[str] = None,
        first: Optional[Runnable[Any, Any]] = None,
        middle: Optional[list[Runnable[Any, Any]]] = None,
        last: Optional[Runnable[Any, Any]] = None,
    ) -> None:
        ...
```
因为存在 `Optional`，所以是可以嵌套的，如果超出了 first, middle, last，那就在此基础上嵌套再一层。然后再程序中递归运行。

接下来，我们尝试结合 LLM 去使用 chain。

In [28]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,  # 让我们的 LLM 最准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

In [30]:
from langchain_core.output_parsers.json import SimpleJsonOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template='In JSON format, I will give you a sentence in English: {sentence} and please translate in a Cat, Dog, Wolf, Lion, Duck and Pig language.')

chain = prompt | llm | SimpleJsonOutputParser()

chain.invoke({'sentence': 'Hello, world!'})  # noqa: T201

{'Cat': 'Meow, purr!',
 'Dog': 'Woof, bark!',
 'Wolf': 'Howl, growl!',
 'Lion': 'Roar, grrr!',
 'Duck': 'Quack, quack!',
 'Pig': 'Oink, snort!'}

接下来我们介绍 `RunnableParallel`，这是一个 composition primitive，允许同时运行多个可运行对象，并为每个 `Runnable` 提供相同的输入。在传统链中，如果你有多个工具要调用，是顺序执行的（一个一个来），效率低。但是在 LangChain 中允许你并行执行多个子链，大幅减少响应延迟（latency），可参考[官方文档](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html#runnableparallel)。以下例子仅作参考：

```python
from langchain_core.runnables import RunnableParallel

google_search_chain = prompt | llm | "GOOGLE SEARCH"
bing_search_chain = prompt | llm | "BING SEARCH"

parallel_chain = RunnableParallel({
    "google": google_search_chain,
    "bing": bing_search_chain,
    # ...
})
```

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

joke_chain = (
        ChatPromptTemplate.from_template("tell me a joke about {topic}")
        | llm
)
poem_chain = (
        ChatPromptTemplate.from_template("write a 4-line poem about {topic}")
        | llm
)

runnable = RunnableParallel(joke=joke_chain, poem=poem_chain)

response = runnable.invoke({"topic": "tiger"})
print("Joke:", response['joke'].content)
print("Poem:", response['poem'].content)
  # noqa: T201

Joke: Sure, here's a tiger joke for you:

Why don't tigers like fast food?

Because they can't catch it!
Poem: In the jungle's heart, a fierce king roams,  
With eyes like fire and silent, stealthy tones.  
Striped with power, grace in every stride,  
In the shadowed forest, the tiger abides.
